In [0]:
tiers = ["bronze","silver","gold"]

adls_paths = { tier : f"abfss://{tier}@epdatalake123.dfs.core.windows.net/" for tier in tiers }

bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"]

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)



In [0]:
import requests
import json
from datetime import timedelta, date
from pyspark.sql.functions import *
from pyspark.sql.types import * 

In [0]:
start_date = date.today() - timedelta(1)
end_date = date.today()

In [0]:
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"


try:
    response = requests.get(url)
    response.raise_for_status()  
    data = response.json().get('features', [])

    if not data: 
         print("No data returned for the specified date range")
    else:
        
        file_path = f"{bronze_adls}{start_date}_earthquake_data.json"
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"data sucessfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"error fetchig data from API:{e}")        

In [0]:
data[0]

{'type': 'Feature',
 'properties': {'mag': -0.61,
  'place': '83 km NNW of Karluk, Alaska',
  'time': 1743119514350,
  'updated': 1743139564820,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/av93037249',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=av93037249&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 0,
  'net': 'av',
  'code': '93037249',
  'ids': ',av93037249,',
  'sources': ',av,',
  'types': ',origin,phase-data,',
  'nst': 7,
  'dmin': 0.05923,
  'rms': 0.09,
  'gap': 105,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M -0.6 - 83 km NNW of Karluk, Alaska'},
 'geometry': {'type': 'Point',
  'coordinates': [-155.112333333333, 58.2336666666667, 3.27]},
 'id': 'av93037249'}

In [0]:
output_data = {"start_date" : start_date.isoformat(),
               "end_date" : end_date.isoformat(),
               "bronze_adls" : bronze_adls,
               "silver_adls" : silver_adls,
               "gold_adls" : gold_adls
}

dbutils.jobs.taskValues.set(key = "bronze_output", value = output_data)